In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from stree import Stree

In [2]:
#X, y = load_iris(return_X_y=True)
#y[y==2] = 0

In [3]:
import pandas as pd
df = pd.read_csv('data/creditcard.csv')
y = df.Class.values
X = df.drop(['Class', 'Time', 'Amount'], axis=1).values

In [4]:
c = Stree(C=17, max_depth=2)
print(c)
c.fit(X, y)
print(len(list(c)))
print(c)

In [5]:
#'base_estimator': [DecisionTreeClassifier(max_depth=1), Stree(max_depth=2), Stree(max_depth=3)],
parameters = {
    'base_estimator': [Stree(max_depth=2), Stree(max_depth=3)],
    'n_estimators': [20, 50, 100, 150],
    'learning_rate': [.5, 1, 1.5] 
}

In [6]:
#parameters = {
#    'base_estimator': [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=5), Stree(), Stree(C=.1), Stree(C=.01), Stree(C=3)],
#    'n_estimators': [20, 50, 100, 150],
#    'learning_rate': [.5, 1, 1.5]           
#}

In [7]:
random_state=2020
clf = AdaBoostClassifier(random_state=random_state)
grid = GridSearchCV(clf, parameters, verbose=10, n_jobs=-1, return_train_score=True)
grid.fit(X, y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1837s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0313s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0302s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  92 out of 120 | elapsed:    2.7s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done 118 out of 120 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=

ValueError: Stree doesn't support sample_weight.

In [8]:
print(grid.best_estimator_)

AdaBoostClassifier(base_estimator=Stree(max_depth=2), learning_rate=0.5,
                   n_estimators=20, random_state=2020)
